In [1]:
!pip install cirq

     |████████████████████████████████| 1.4MB 4.5MB/s 
     |████████████████████████████████| 1.2MB 5.7MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


In [2]:
import cirq

q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(1, 0)

def basic_circuit(meas=True):
    sqrt_x = cirq.X**0.5
    yield sqrt_x(q0), sqrt_x(q1)
    yield cirq.CZ(q0, q1)
    yield sqrt_x(q0), sqrt_x(q1)
    if meas:
        yield cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')

circuit = cirq.Circuit()
circuit.append(basic_circuit())

print(circuit)

(0, 0): ───X^0.5───@───X^0.5───M('q0')───
                   │
(1, 0): ───X^0.5───@───X^0.5───M('q1')───


In [3]:
from cirq import Simulator
simulator = Simulator()
result = simulator.run(circuit)

print(result)

q0=0
q1=1


In [4]:
result = simulator.run(circuit)

print(result)

q0=1
q1=1


In [5]:
import numpy as np
circuit = cirq.Circuit()
circuit.append(basic_circuit(False))
result = simulator.simulate(circuit, qubit_order=[q0, q1])

print(np.around(result.final_state, 3))

[0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]


In [6]:
outside = [1, 10]
inside = [1, 2]
print(np.kron(outside, inside))

[ 1  2 10 20]


In [7]:
i = 0
for first in [0, 1]:
    for second in [0, 1]:
        print('amps[{}] is for first={}, second={}'.format(i, first, second))
        i += 1

amps[0] is for first=0, second=0
amps[1] is for first=0, second=1
amps[2] is for first=1, second=0
amps[3] is for first=1, second=1


In [0]:
q_stay = cirq.NamedQubit('q_stay')
q_flip = cirq.NamedQubit('q_flip')
c = cirq.Circuit(cirq.X(q_flip))

In [11]:
# first qubit in order flipped
result = simulator.simulate(c, qubit_order=[q_flip, q_stay])
print(abs(result.final_state).round(3))

[0. 0. 1. 0.]


In [12]:
# second qubit in order flipped
result = simulator.simulate(c, qubit_order=[q_stay, q_flip])
print(abs(result.final_state).round(3))

[0. 1. 0. 0.]


In [13]:
circuit = cirq.Circuit()
circuit.append(basic_circuit())
for i, step in enumerate(simulator.simulate_moment_steps(circuit)):
    print('state at step %d: %s' % (i, np.around(step.state_vector(), 3)))

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [0.+0.j 0.+0.j 0.+1.j 0.+0.j]


In [14]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.bit_flip(p=0.2)(q), cirq.measure(q))
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='a'))

Counter({0: 77, 1: 23})


In [15]:
import sympy
rot_w_gate = cirq.X**sympy.Symbol('x')
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
for y in range(5):
    resolver = cirq.ParamResolver({'x': y / 4.0})
    result = simulator.simulate(circuit, resolver)
    print(np.round(result.final_state, 2))

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[ 0.6 +0.6j   0.25-0.25j  0.25-0.25j -0.1 -0.1j ]
[0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
[-0.1 +0.1j   0.25+0.25j  0.25+0.25j  0.6 -0.6j ]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]


In [20]:
resolvers = [cirq.ParamResolver({'x': y / 2.0}) for y in range(3)]
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
circuit.append([cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')])
results = simulator.run_sweep(program=circuit,
                              params=resolvers,
                              repetitions=20)
for result in results:
    print(result)

q0=00000000000000000000
q1=00000000000000000000
q0=10100100110010000010
q1=00101010111111000100
q0=11111111111111111111
q1=11111111111111111111


In [21]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q), cirq.measure(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='a'))

Counter({0: 58, 1: 42})


In [25]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit(cirq.H(q), cirq.amplitude_damp(0.2)(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.simulate(circuit)
print(np.around(result.final_density_matrix, 3))

[[0.6  +0.j 0.447+0.j]
 [0.447+0.j 0.4  +0.j]]
